In [1]:
import pandas as pd
import wandb
from tqdm.notebook import tqdm
import pickle
from os.path import exists
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import math
from matplotlib.ticker import MaxNLocator

from data.get_uci import all_datasets
from analysis.util import fetch, init_uci_dict, get_uci_info

In [2]:
filters = {
    "group": "benchmark5"
}
raw = fetch("soft-gp-2", filters)

139it [00:59,  2.32it/s]                         


In [3]:
uci_info = get_uci_info()

In [7]:
uci_dict = {}
for exp in raw:
    model = exp.config["model.name"]
    dataset = exp.config["dataset.name"]
    num_inducing = exp.config["model.num_inducing"]
    dtype = exp.config["model.dtype"]
    seed = exp.config["training.seed"]
    train_frac = float(exp.config["dataset.train_frac"])
    if model == "svi-gp" and not exp.config["model.learn_noise"]:
        continue 
    if model == "sv-gp" and not exp.config["model.learn_noise"]:
        continue 
    # if np.allclose(0.4444444444444444, train_frac):
    #     train_frac = 0.44
    # elif np.allclose(0.888888888888, train_frac):
    #     train_frac = 0.89
    uci_dict[(dataset, seed, num_inducing, train_frac, model)] = exp.history

print(uci_dict.keys())

dict_keys([('keggundirected', 6535, 512, 0.9, 'sv-gp'), ('3droad', 8830, 512, 0.9, 'svi-gp'), ('slice', 92357, 1024, 0.9, 'sv-gp'), ('keggundirected', 6535, 512, 0.9, 'soft-gp'), ('slice', 92357, 1024, 0.9, 'soft-gp'), ('slice', 92357, 512, 0.9, 'sv-gp'), ('slice', 8830, 1024, 0.9, 'sv-gp'), ('slice', 92357, 512, 0.9, 'soft-gp'), ('slice', 8830, 512, 0.9, 'sv-gp'), ('slice', 8830, 1024, 0.9, 'soft-gp'), ('slice', 6535, 1024, 0.9, 'sv-gp'), ('slice', 8830, 512, 0.9, 'soft-gp'), ('slice', 6535, 512, 0.9, 'sv-gp'), ('slice', 6535, 1024, 0.9, 'soft-gp'), ('keggdirected', 92357, 1024, 0.9, 'sv-gp'), ('slice', 6535, 512, 0.9, 'soft-gp'), ('keggdirected', 92357, 512, 0.9, 'sv-gp'), ('3droad', 6535, 1024, 0.9, 'svi-gp'), ('keggdirected', 92357, 1024, 0.9, 'soft-gp'), ('keggdirected', 8830, 1024, 0.9, 'sv-gp'), ('keggdirected', 92357, 512, 0.9, 'soft-gp'), ('keggdirected', 8830, 512, 0.9, 'sv-gp'), ('keggdirected', 8830, 1024, 0.9, 'soft-gp'), ('keggundirected', 92357, 1024, 0.9, 'svi-gp'), ('k

In [10]:
seeds = [6535, 8830, 92357]
num_inducings = [512, 1024]
fracs = [0.9]
# fracs = [0.44, 0.89]
tmp = {
    "N": [int(np.floor(N * 0.9)) for _, N, _, _ in uci_info],
    "D": [D for _, _, D, _ in uci_info],
    "exact-rep": [e for _, _, _, e in uci_info],
}
models = ["soft-gp", "svi-gp", "sv-gp"]

# models = ["sv-gp"]
for seed in seeds:
    for model in models:
        for num_inducing in num_inducings:
            for frac in fracs:
                xs = []
                ts = []
                for dataset, _, _, _ in uci_info:
                    try:
                        xs += [uci_dict[(dataset, seed, num_inducing, frac, model)]["test_rmse"][49]]
                        ts += [np.array(uci_dict[(dataset, seed, num_inducing, frac, model)]["epoch_time"][49]).mean()]
                    except Exception as e:
                        xs += [np.nan]
                        ts += [np.nan]
                        print("Exception", e, model, dataset)

                tmp[f"{model}-{num_inducing}-{frac}-{seed}"] = xs
                tmp[f"time-{model}-{num_inducing}-{frac}-{seed}"] = ts
df = pd.DataFrame(data=tmp)
df.index = [name.capitalize().replace("_", "-") for name, _, _, _ in uci_info]
df

Exception ('3droad', 6535, 512, 0.9, 'soft-gp') soft-gp 3droad
Exception ('song', 6535, 512, 0.9, 'soft-gp') soft-gp song
Exception ('buzz', 6535, 512, 0.9, 'soft-gp') soft-gp buzz
Exception ('houseelectric', 6535, 512, 0.9, 'soft-gp') soft-gp houseelectric
Exception ('keggundirected', 6535, 1024, 0.9, 'soft-gp') soft-gp keggundirected
Exception ('3droad', 6535, 1024, 0.9, 'soft-gp') soft-gp 3droad
Exception ('song', 6535, 1024, 0.9, 'soft-gp') soft-gp song
Exception ('buzz', 6535, 1024, 0.9, 'soft-gp') soft-gp buzz
Exception ('houseelectric', 6535, 1024, 0.9, 'soft-gp') soft-gp houseelectric
Exception ('song', 6535, 512, 0.9, 'svi-gp') svi-gp song
Exception ('buzz', 6535, 512, 0.9, 'svi-gp') svi-gp buzz
Exception ('houseelectric', 6535, 512, 0.9, 'svi-gp') svi-gp houseelectric
Exception 49 svi-gp 3droad
Exception ('song', 6535, 1024, 0.9, 'svi-gp') svi-gp song
Exception ('buzz', 6535, 1024, 0.9, 'svi-gp') svi-gp buzz
Exception ('houseelectric', 6535, 1024, 0.9, 'svi-gp') svi-gp housee

,N,D,exact-rep,soft-gp-512-0.9-6535,time-soft-gp-512-0.9-6535,soft-gp-1024-0.9-6535,time-soft-gp-1024-0.9-6535,svi-gp-512-0.9-6535,time-svi-gp-512-0.9-6535,svi-gp-1024-0.9-6535,...,soft-gp-1024-0.9-92357,time-soft-gp-1024-0.9-92357,svi-gp-512-0.9-92357,time-svi-gp-512-0.9-92357,svi-gp-1024-0.9-92357,time-svi-gp-1024-0.9-92357,sv-gp-512-0.9-92357,time-sv-gp-512-0.9-92357,sv-gp-1024-0.9-92357,time-sv-gp-1024-0.9-92357
Pol,13500,26,0.151,0.188628,1.078404,0.189350,0.986293,0.309099,1.130732,0.294593,...,0.198989,1.072062,0.324796,1.192581,0.303140,1.301662,0.265854,0.011047,0.252374,0.014232
Elevators,14939,18,0.394,0.394229,1.203841,0.389570,1.287340,0.393341,1.300659,0.391176,...,0.382464,1.235404,0.383668,1.246262,0.382765,1.430118,0.394750,0.012915,0.390247,0.013604
Bike,15641,17,0.220,0.205709,1.235199,0.214159,1.249359,0.280584,1.301765,0.267323,...,0.208200,1.356058,0.278015,1.283305,0.266501,1.553516,0.282527,0.009482,0.269968,0.014083
Kin40k,36000,8,0.099,0.243087,2.831931,0.237758,2.843454,0.239725,3.010290,0.198606,...,0.236304,2.928761,0.230655,2.973364,0.193320,3.410730,0.201698,0.010503,0.171724,0.019196
Protein,41157,9,0.536,0.660346,3.205755,0.658084,3.384262,0.691128,3.676847,0.666071,...,0.652494,3.294114,0.686090,3.555199,0.658803,4.031876,0.662626,0.013809,0.630434,0.020502
Keggdirected,43944,20,0.086,0.079015,3.603906,0.079483,3.494276,0.086545,3.891691,0.086298,...,0.086798,3.479722,0.094188,3.691482,0.093710,4.392901,0.116749,0.012023,0.096646,0.019436
Slice,48150,385,0.262,0.048065,3.878796,0.042071,4.558107,0.142447,4.316448,0.129656,...,0.051239,4.346431,0.135433,4.211383,0.124283,4.939213,0.454741,0.014103,0.408399,0.024008
Keggundirected,57247,27,0.118,0.115877,4.735317,NaN,NaN,0.131222,4.944376,0.130906,...,NaN,NaN,0.131012,6.328258,0.130714,5.538306,NaN,NaN,NaN,NaN
3droad,391386,3,0.101,NaN,NaN,NaN,NaN,0.530587,33.560304,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Song,270000,90,0.807,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
def pm_var(df, model):
    print(df[[f'{model}-6535', f'{model}-8830', f'{model}-92357']].mean(axis=1))
    m = df[[f'{model}-6535', f'{model}-8830', f'{model}-92357']].mean(axis=1).round(3).astype(str)
    v = df[[f'{model}-6535', f'{model}-8830', f'{model}-92357']].std(axis=1).round(3).astype(str).apply(lambda x: f" $\pm$ {x}")
    return (m + v).apply(lambda x: x.replace("nan $\pm$ nan", "-"))
df_rmse = pd.DataFrame()
df_rmse[['N', 'D', 'exact-rep']] = df[['N', 'D', 'exact-rep']]
for model in models:
    for num_inducing in num_inducings:
        for frac in fracs:
            df_rmse[f'{model}-{num_inducing}-{frac}'] = pm_var(df, f'{model}-{num_inducing}-{frac}')
            # df_rmse['svi-gp'] = pm_var(df, "svi-gp")
            # df_rmse['soft-gp'] = pm_var(df, "soft-gp")

df_rmse

                soft-gp-512-0.9-6535 soft-gp-512-0.9-8830  \
Pol                         0.188628             0.199472   
Elevators                   0.394229             0.395359   
Bike                        0.205709             0.207337   
Kin40k                      0.243087                  NaN   
Protein                     0.660346             0.637054   
Keggdirected                0.079015             0.078073   
Slice                       0.048065             0.046784   
Keggundirected              0.115877                  NaN   
3droad                           NaN                  NaN   
Song                             NaN                  NaN   
Buzz                             NaN                  NaN   
Houseelectric                    NaN                  NaN   

                soft-gp-512-0.9-92357  
Pol                          0.200360  
Elevators                    0.377419  
Bike                         0.195738  
Kin40k                       0.243482  
Protei

TypeError: unsupported operand type(s) for +: 'float' and 'str'

In [ ]:
df2 = df_rmse[["N", "D", "exact-rep", "soft-gp-512-0.44", "sv-gp-512-0.44", "svi-gp-512-0.44", "svi-gp-512-0.44"]]
df2 = df2.sort_values(by=['D'], ascending=[True])
df2

In [ ]:
df3 = df_rmse[["N", "D", "exact-rep", "soft-gp-1024-0.44", "sv-gp-1024-0.44", "svi-gp-1024-0.44"]]
df3 = df2.sort_values(by=['D'], ascending=[True])
df3

In [ ]:
df4 = df_rmse[["N", "D", "exact-rep", "soft-gp-512-0.89", "sv-gp-512-0.89", "svi-gp-512-0.89"]]
df4 = df4.sort_values(by=['D'], ascending=[True])
df4

In [ ]:
df4 = df_rmse[["N", "D", "exact-rep", "soft-gp-1024-0.89", "sv-gp-1024-0.89", "svi-gp-1024-0.89"]]
df4 = df4.sort_values(by=['D'], ascending=[True])
df4

In [ ]:
latex_table = df_rmse.to_latex(
    index=True,
    escape=False,
    float_format="{:0.3f}".format,
)
print(latex_table)

In [ ]:
def pm_var_time(df, model):
    m = df[[f'time-{model}-6535', f'time-{model}-8830', f'time-{model}-42']].mean(axis=1).round(3).astype(str)
    v = df[[f'time-{model}-6535', f'time-{model}-8830', f'time-{model}-42']].std(axis=1).round(3).astype(str).apply(lambda x: f" $\pm$ {x}")
    return (m + v).apply(lambda x: x.replace("nan $\pm$ nan", "-"))

tmp = {
    "N": [int(np.floor(N * 4/9)) for _, N, _, _ in uci_info],
    "D": [D for _, _, D, _ in uci_info],
    "exact-rep": [e for _, _, _, e in uci_info],
}
df_time = pd.DataFrame()
df_time.index = [name.capitalize().replace("_", "-") for name, _, _, _ in uci_info]
for model in models:
    for num_inducing in num_inducings:
        for frac in fracs:
            col = pm_var_time(df, f'{model}-{num_inducing}-{frac}')
            df_time[f'{model}-{num_inducing}-{frac}'] = col
# # df_time["exact"] = pm_var_time(df, "exact")
# # df_time["sv-gp"] = pm_var_time(df, "sv-gp")
# df_time["svi-gp"] = pm_var_time(df, "svi-gp")
# df_time["soft-gp"] = pm_var_time(df, "soft-gp")
df_time

In [ ]:
tmp = {
    "N": [int(np.floor(N * 4/9)) for _, N, _, _ in uci_info],
    "D": [D for _, _, D, _ in uci_info],
    "exact-rep": [e for _, _, _, e in uci_info],
}
df_44 = pd.DataFrame(data=tmp)
df_44.index = [name.capitalize().replace("_", "-") for name, _, _, _ in uci_info]
df5 = pd.concat([df_44, df_time[["soft-gp-1024-0.44", "sv-gp-1024-0.44", "svi-gp-1024-0.44"]]], axis=1)
df5 = df5.sort_values(by=['D'], ascending=[True])
df5

In [ ]:
tmp = {
    "N": [int(np.floor(N * 8/9)) for _, N, _, _ in uci_info],
    "D": [D for _, _, D, _ in uci_info],
    "exact-rep": [e for _, _, _, e in uci_info],
}
df_89 = pd.DataFrame(data=tmp)
df_89.index = [name.capitalize().replace("_", "-") for name, _, _, _ in uci_info]
df5 = pd.concat([df_89, df_time[["soft-gp-1024-0.89", "sv-gp-1024-0.89", "svi-gp-1024-0.89"]]], axis=1)
df5 = df5.sort_values(by=['D'], ascending=[True])
df5

In [ ]:

latex_table = df_time.to_latex(
    index=True,
    escape=False,
    float_format="{:0.3f}".format,
)
print(latex_table)